# Training and Detecting Object with GeoAI


## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [ ]:
%pip install geoai-py segment-geospatial

## Import libraries

In [1]:
import geoai
import leafmap
from samgeo import tms_to_geotiff, split_raster

## Download satellite

Create an interactive map

In [3]:
m = leafmap.Map(center=[24.699273257355753, 46.68045811333872], zoom=17, height="800px")
m.add_basemap("SATELLITE")
m

Map(center=[24.699273257355753, 46.68045811333872], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Create boundingbox and download image

In [4]:
m.user_roi

{'type': 'Feature',
 'properties': {},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[39.815526, 21.435373],
    [39.815526, 21.441105],
    [39.824131, 21.441105],
    [39.824131, 21.435373],
    [39.815526, 21.435373]]]}}

In [5]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [46.66867, 24.70191, 46.68044, 24.6916]

image = "data\Satellite\Makkah.tif"
# tms_to_geotiff(output=image, bbox=bbox, zoom=21, source="Satellite", overwrite=True)   
leafmap.map_tiles_to_geotiff(
    output=image, bbox=bbox, zoom=19, source="Satellite", overwrite=True
) 

Downloaded image 1/140
Downloaded image 2/140
Downloaded image 3/140
Downloaded image 4/140
Downloaded image 5/140
Downloaded image 6/140
Downloaded image 7/140
Downloaded image 8/140
Downloaded image 9/140
Downloaded image 10/140
Downloaded image 11/140
Downloaded image 12/140
Downloaded image 13/140
Downloaded image 14/140
Downloaded image 15/140
Downloaded image 16/140
Downloaded image 17/140
Downloaded image 18/140
Downloaded image 19/140
Downloaded image 20/140
Downloaded image 21/140
Downloaded image 22/140
Downloaded image 23/140
Downloaded image 24/140
Downloaded image 25/140
Downloaded image 26/140
Downloaded image 27/140
Downloaded image 28/140
Downloaded image 29/140
Downloaded image 30/140
Downloaded image 31/140
Downloaded image 32/140
Downloaded image 33/140
Downloaded image 34/140
Downloaded image 35/140
Downloaded image 36/140
Downloaded image 37/140
Downloaded image 38/140
Downloaded image 39/140
Downloaded image 40/140
Downloaded image 41/140
Downloaded image 42/140
D

## Load training data

In [6]:
train_raster_path = r"data\POC3\Roi_Jun16.tif"
train_vector_path = r"data\POC3\Labels50.geojson"

## Visualize sample data

In [ ]:
geoai.view_vector_interactive(train_vector_path, tiles=train_raster_path)

## Create training data

In [ ]:
out_folder = r"results\Satellite\output"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_vector_path,
    tile_size=800,
    stride=400,
    buffer_radius=0,
)

## Train object detection model

In [ ]:
geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models20",
    num_channels=3,
    pretrained=True,
    batch_size=4,
    num_epochs=20,
    learning_rate=0.005,
    val_split=0.2,
)

## Run inference

In [10]:
out_folder = r"results\Hung"
test_raster_path = r"data\Satellite\Makkah.tif"
masks_path = r"results\Hung\Makkak_Satellite_Conf_07_prediction.tif"
model_path = f"{out_folder}/best_model.pth"

In [7]:
geoai.view_raster(test_raster_path)

In [11]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=800,
    overlap=400,
    confidence_threshold=0.7,
    batch_size=4,
    num_channels=3,
)

Processing 40 windows with size 800x800 and overlap 400...


54it [01:58,  2.19s/it]                        

Inference completed in 118.19 seconds
Saved prediction to results\Hung\Makkak_Satellite_Conf_07_prediction.tif


In [12]:
# masks_path = r"results\POC3\120Epochs\Test3_120_Jun16_prediction.tif"
output_path = r"results\Hung\Makkak_Satellite_Conf_07_prediction.geojson"
gdf = geoai.orthogonalize(masks_path, output_path, epsilon=2)

Processing 676 features...


Converting features: 100%|██████████| 676/676 [00:06<00:00, 99.07shape/s] 


Saving to results\Hung\Makkak_Satellite_Conf_07_prediction.geojson...
Done!


## Visualize results

In [ ]:
geoai.view_vector_interactive(output_path, tiles=test_raster_path)